In [2]:
##### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/annotated-dataset-for-knee-arthritis-detection/Training/1Doubtful/DoubtfulG1 (76).png
/kaggle/input/annotated-dataset-for-knee-arthritis-detection/Training/1Doubtful/DoubtfulG1 (192).png
/kaggle/input/annotated-dataset-for-knee-arthritis-detection/Training/1Doubtful/DoubtfulG1 (352).png
/kaggle/input/annotated-dataset-for-knee-arthritis-detection/Training/1Doubtful/DoubtfulG1 (432).png
/kaggle/input/annotated-dataset-for-knee-arthritis-detection/Training/1Doubtful/DoubtfulG1 (379).png
/kaggle/input/annotated-dataset-for-knee-arthritis-detection/Training/1Doubtful/DoubtfulG1 (101).png
/kaggle/input/annotated-dataset-for-knee-arthritis-detection/Training/1Doubtful/DoubtfulG1 (16).png
/kaggle/input/annotated-dataset-for-knee-arthritis-detection/Training/1Doubtful/DoubtfulG1 (200).png
/kaggle/input/annotated-dataset-for-knee-arthritis-detection/Training/1Doubtful/DoubtfulG1 (373).png
/kaggle/input/annotated-dataset-for-knee-arthritis-detection/Training/1Doubtful/DoubtfulG1 (3

In [134]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch import nn

In [135]:
transform=transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    #transforms.RandomRotation(180),
    #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.Resize((128,128)),
    transforms.ToTensor(),#Tensors are the standard data structure used in PyTorch for modeling.
    transforms.Normalize((0.5,),(0.5,))
])

In [136]:
root='/kaggle/input/annotated-dataset-for-knee-arthritis-detection/Training'
dataset=datasets.ImageFolder(root=root,transform=transform)

In [137]:
train_size=int(0.80*(len(dataset)))
test_size=len(dataset)-train_size
train_dataset,test_dataset=random_split(dataset,[train_size,test_size])

In [138]:
class_names=dataset.classes
class_idx=dataset.class_to_idx
print(class_names,class_idx)

['0Normal', '1Doubtful', '2Mild', '3Moderate', '4Severe'] {'0Normal': 0, '1Doubtful': 1, '2Mild': 2, '3Moderate': 3, '4Severe': 4}


In [139]:
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=True)

In [147]:
class CNN(nn.Module):
    def __init__(self, inp_size, no_of_classes):
        super(CNN, self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(128 * (inp_size // 8) * (inp_size // 8), 256),  # Assuming input_size is divisible by 8
            nn.ReLU(),
            nn.Linear(256, no_of_classes)
        )        

    def forward(self, x):
        return self.network(x)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [148]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
inp_size=128
num_epochs=50
model = CNN(inp_size=inp_size, no_of_classes=len(class_names))
model = model.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [149]:
for epoch in range(num_epochs):
    for inp,output in train_loader:
        inp,output = inp.to(device),output.to(device)
        pred_output=model(inp)
        loss=criterion(pred_output,output)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')
    if loss.item() <0.01:
        break

Epoch 1/50, Loss: 1.5744
Epoch 2/50, Loss: 1.9219
Epoch 3/50, Loss: 1.3036
Epoch 4/50, Loss: 0.7964
Epoch 5/50, Loss: 1.3898
Epoch 6/50, Loss: 1.0325
Epoch 7/50, Loss: 0.8786
Epoch 8/50, Loss: 1.1895
Epoch 9/50, Loss: 0.6585
Epoch 10/50, Loss: 0.9004
Epoch 11/50, Loss: 1.1145
Epoch 12/50, Loss: 0.4974
Epoch 13/50, Loss: 1.4531
Epoch 14/50, Loss: 0.4333
Epoch 15/50, Loss: 0.4682
Epoch 16/50, Loss: 0.9211
Epoch 17/50, Loss: 0.9944
Epoch 18/50, Loss: 0.6244
Epoch 19/50, Loss: 0.7102
Epoch 20/50, Loss: 0.6532
Epoch 21/50, Loss: 1.0152
Epoch 22/50, Loss: 0.7367
Epoch 23/50, Loss: 0.5611
Epoch 24/50, Loss: 0.2625
Epoch 25/50, Loss: 0.5310
Epoch 26/50, Loss: 0.1474
Epoch 27/50, Loss: 0.4458
Epoch 28/50, Loss: 1.0020
Epoch 29/50, Loss: 0.2736
Epoch 30/50, Loss: 0.6010
Epoch 31/50, Loss: 0.5324
Epoch 32/50, Loss: 0.1005
Epoch 33/50, Loss: 0.3280
Epoch 34/50, Loss: 0.8468
Epoch 35/50, Loss: 0.0828
Epoch 36/50, Loss: 0.1829
Epoch 37/50, Loss: 0.4278
Epoch 38/50, Loss: 0.6671
Epoch 39/50, Loss: 0.

In [150]:


# Placeholders for predictions and actual labels
y_pred, y_test = [], []

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation
with torch.no_grad():
    for inp, output in test_loader:
        inp, output = inp.to(device), output.to(device)
        pred_output = model(inp)
        
        # Move tensors to CPU before converting to NumPy arrays
        y_pred.extend(torch.argmax(pred_output, dim=1).cpu().numpy())
        y_test.extend(output.cpu().numpy())

# Switch the model back to training mode
model.train()

# Calculate true positives for each class
t1c = sum(1 for yp, yt in zip(y_pred, y_test) if yp == 0 and yt == 0)
t2c = sum(1 for yp, yt in zip(y_pred, y_test) if yp == 1 and yt == 1)
t3c = sum(1 for yp, yt in zip(y_pred, y_test) if yp == 2 and yt == 2)
t4c = sum(1 for yp, yt in zip(y_pred, y_test) if yp == 3 and yt == 3)
t5c = sum(1 for yp, yt in zip(y_pred, y_test) if yp == 4 and yt == 4)

# Calculate total instances of each class in the test set
t1 = sum(1 for yt in y_test if yt == 0)
t2 = sum(1 for yt in y_test if yt == 1)
t3 = sum(1 for yt in y_test if yt == 2)
t4 = sum(1 for yt in y_test if yt == 3)
t5 = sum(1 for yt in y_test if yt == 4)

# Print the results
print('label 1={}/{}, label 2={}/{}, label 3={}/{}, label 4={}/{}, label 5={}/{}'.format(t1c, t1, t2c, t2, t3c, t3, t4c, t4, t5c, t5))


label 1=60/84, label 2=58/97, label 3=25/55, label 4=43/50, label 5=35/44


In [151]:
def accuracy(loader, model, device):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            scores = model(x)
            _,predictions=scores.max(1)
            num_correct+=(predictions==y).sum().item()
            num_samples+=predictions.size(0)
    model.train()
    return num_correct / num_samples

# Ensure the model is on the correct device
model = model.to(device)

# Print training and testing accuracy
print(f'Training Accuracy: {accuracy(train_loader, model, device)*100:.2f}')
print(f'Testing Accuracy: {accuracy(test_loader, model, device)*100:.2f}')


Training Accuracy: 97.20
Testing Accuracy: 70.61


In [152]:
saved_path = "/kaggle/working/saved_path.pth"
torch.save(model.state_dict(), saved_path)